# code

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "/gpu:0"


In [2]:
# Loading and preprocessing data
text_data_path = 'modified_text.csv'
image_metadata_path = 'indiana_projections.csv'
image_folder_path = 'images/images_normalized'

# Load text data
text_data = pd.read_csv(text_data_path)

# Load image metadata
image_metadata = pd.read_csv(image_metadata_path)
image_metadata['filename'] = image_metadata['filename'].apply(lambda x: os.path.join(image_folder_path, x))

# Merge text and image data
merged_data = pd.merge(text_data, image_metadata[['uid', 'filename']], on='uid')

# Save the merged data
# merged_data.to_csv('merged_data.csv', index=False)

In [3]:
# Create a Tokenizer and process text data
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(merged_data['notes'])
sequences = tokenizer.texts_to_sequences(merged_data['notes'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

# Define image preprocessing function
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(target_size)
    image_array = np.array(image)
    from tensorflow.keras.applications.resnet50 import preprocess_input
    image_array = preprocess_input(image_array)
    print(image_array.shape)
    return image_array

# Load and preprocess all images
images = np.array([load_and_preprocess_image(img_path) for img_path in merged_data['filename']])


(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

In [4]:
images_file_path = 'images.npy'
np.save(images_file_path, images)

In [5]:
images = np.load(images_file_path)
images.shape

(7466, 224, 224, 3)

# Handling Labels

In [6]:
# Handling Labels
merged_data['labels'] = merged_data['labels'].fillna('')

def process_labels(label):
    if isinstance(label, list):
        return label
    if isinstance(label, str) and label != '':
        try:
            return list(map(int, label.split(';')))
        except ValueError:
            return []
    return []

labels = merged_data['labels'].apply(process_labels)
mlb = MultiLabelBinarizer(classes=list(range(14)))
labels = mlb.fit_transform(labels)

# Ensure data length is consistent
assert len(padded_sequences) == len(images) == len(labels), "The data length is inconsistent!"

In [7]:
# Convert to NumPy array
padded_sequences = np.array(padded_sequences)
images = np.array(images)
labels = np.array(labels)

In [8]:
images.shape

(7466, 224, 224, 3)

In [9]:
labels.shape,padded_sequences.shape

((7466, 14), (7466, 100))

# Define a new accuracy function

In [10]:
import tensorflow as tf

class CustomAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_accuracy', **kwargs):
        super(CustomAccuracy, self).__init__(name=name, **kwargs)
        self.total_correct = self.add_weight(name='total_correct', initializer='zeros', dtype=tf.float32)
        self.total_labels = self.add_weight(name='total_labels', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):

        # Convert the prediction results to binary
        y_pred_binary = tf.cast(y_pred > 0.5, tf.float32)

        # Condition 1: Label 0 is predicted as positive
        condition1 = y_pred_binary[:, 0] > 0.5  # shape: (batch_size,)

        # Condition 2: Any of the other labels is predicted to be positive
        condition2 = tf.reduce_any(y_pred_binary[:, 1:] > 0.5, axis=1)  # shape: (batch_size,)

        # Step 1: If condition 1 is true, set other labels to 0
        condition1_expanded = tf.expand_dims(condition1, axis=1)  # shape: (batch_size, 1)
        y_pred_modified = tf.where(
            condition1_expanded,
            tf.concat([y_pred_binary[:, 0:1], tf.zeros_like(y_pred_binary[:, 1:])], axis=1),
            y_pred_binary
        )

        # Step 2: If condition 1 is false and condition 2 is true, set label 0 to 0
        condition2_only = tf.logical_and(tf.logical_not(condition1), condition2)  # shape: (batch_size,)
        condition2_only_expanded = tf.expand_dims(condition2_only, axis=1)  # shape: (batch_size, 1)
        y_pred_modified = tf.where(
            condition2_only_expanded,
            tf.concat([tf.zeros_like(y_pred_binary[:, 0:1]), y_pred_modified[:, 1:]], axis=1),
            y_pred_modified
        )

        # Compute the correct prediction for each label
        correct_predictions = tf.cast(tf.equal(y_pred_modified, y_true), tf.float32)

        # Calculate the total number of correct predictions for this batch
        correct_sum = tf.reduce_sum(correct_predictions)

        # Calculate the total number of labels in this batch
        labels_sum = tf.cast(tf.size(y_true), tf.float32)

        # Update the cumulative number of correct predictions and total number of labels
        self.total_correct.assign_add(correct_sum)
        self.total_labels.assign_add(labels_sum)

    def result(self):
        """
        Returns the accuracy, guaranteed to be between 0 and 1.
        """
        return self.total_correct / self.total_labels

    def reset_state(self):
        """
        Resets the accumulated number of correct predictions and total number of labels.
        """
        self.total_correct.assign(0.0)
        self.total_labels.assign(0.0)


# Split data into multiple clients
### Data shuffle

In [11]:
# Shuffle data
padded_sequences, images, labels = shuffle(padded_sequences, images, labels, random_state=42)
#Function: Shuffle the data to ensure that the data distribution of each client is random.

### Divide data into clients

In [12]:
# Number of clients
num_clients = 3
client_data_size = len(padded_sequences) // num_clients
client_datasets = []
for i in range(num_clients):
    start_index = i * client_data_size
    if i == num_clients - 1:
        end_index = len(padded_sequences)
    else:
        end_index = (i + 1) * client_data_size

    client_padded_sequences = padded_sequences[start_index:end_index]
    client_images = images[start_index:end_index]
    client_labels = labels[start_index:end_index]

    client_datasets.append((client_padded_sequences, client_images, client_labels))

#Function: Distribute data evenly to each client.

# Define Model Structure
### Create Model

In [13]:

def create_model():
    # Text input and processing
    text_input = tf.keras.Input(shape=(100,), name='text_input')
    embedding_layer = tf.keras.layers.Embedding(input_dim=10000, output_dim=128)(text_input)
    lstm_layer = tf.keras.layers.LSTM(128)(embedding_layer)
    text_dense = tf.keras.layers.Dense(128, activation='relu')(lstm_layer)
    text_dropout = tf.keras.layers.Dropout(0.5)(text_dense)
    text_output = tf.keras.layers.Dense(64, activation='relu')(text_dropout)

    # Image input and processing
    image_input = tf.keras.Input(shape=(224, 224, 3), name='image_input')
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)
    base_model.trainable = False
    image_pooling = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    image_dense = tf.keras.layers.Dense(128, activation='relu')(image_pooling)
    image_dropout = tf.keras.layers.Dropout(0.5)(image_dense)
    image_output = tf.keras.layers.Dense(64, activation='relu')(image_dropout)

    # Merge text and image features
    combined = tf.keras.layers.Concatenate()([text_output, image_output])
    combined_dense = tf.keras.layers.Dense(128, activation='relu')(combined)
    combined_dropout = tf.keras.layers.Dropout(0.5)(combined_dense)
    final_output = tf.keras.layers.Dense(14, activation='sigmoid')(combined_dropout)

    model = tf.keras.Model(inputs=[text_input, image_input], outputs=final_output)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    # Using custom CustomAccuracy
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[CustomAccuracy()])

    return model


# Gradient processing module

In [14]:
# Define the function to calculate the gradient
from tqdm import tqdm
import tensorflow as tf
import numpy as np

def compute_gradients(model, data):
    X_text, X_images, y = data
    batch_size = 8
    num_samples = X_text.shape[0]
    gradients_accum = None  # Initialize the gradient accumulator

    for i in tqdm(range(0, num_samples, batch_size), desc="Calculating gradients"):
        batch_X_text = X_text[i:i+batch_size]
        batch_X_images = X_images[i:i+batch_size]
        batch_y = y[i:i+batch_size]

        with tf.GradientTape() as tape:
            predictions = model({'text_input': batch_X_text, 'image_input': batch_X_images}, training=True)
            loss = tf.keras.losses.binary_crossentropy(batch_y, predictions)
            loss = tf.reduce_mean(loss)

        grads = tape.gradient(loss, model.trainable_variables)

        # Convert IndexedSlices to dense tensors
        grads = [tf.convert_to_tensor(g) if isinstance(g, tf.IndexedSlices) else g for g in grads]
        grads = [g if g is not None else tf.zeros_like(v) for g, v in zip(grads, model.trainable_variables)]

        if gradients_accum is None:
            gradients_accum = [np.zeros_like(g.numpy()) for g in grads]

        for idx, g in enumerate(grads):
            gradients_accum[idx] += g.numpy()

    # Calculate the average gradient
    avg_gradients = [g / num_samples for g in gradients_accum]
    return avg_gradients

# Define the function to calculate the gradient change rate
def compute_gradient_change_rate(initial_grads, final_grads):
    total_change = 0
    for initial, final in tqdm(zip(initial_grads, final_grads), desc="Calculating gradient change rate", total=len(initial_grads)):
        numerator = np.linalg.norm(final - initial)
        denominator = np.linalg.norm(initial) + 1e-7
        change = numerator / denominator
        total_change += change
    avg_change = total_change / len(initial_grads)
    return avg_change

#Simulate the federated learning process

In [15]:
# Initialize the global model
global_model = create_model()
# Purpose: Create a global model. Initially, each client will start training from this model.

### Define the number of federated learning rounds

In [16]:
num_rounds = 20#Set the number of iterations of federated learning.

# Start federated learning iteration - follow the gradient

In [18]:
from sklearn.metrics import f1_score, recall_score, precision_score

# Global evaluation metrics
global_metrics = []

for round_num in range(1, num_rounds + 1):
    print(f"\n===== {round_num} round of federated learning started =====")

    #Store weight differences and gradient change rates for all clients
    client_weight_diffs_and_changes = []
    client_metrics = []

    # Store validation data to evaluate the global model
    val_text_list = []
    val_images_list = []
    val_labels_list = []

    # Each client
    for client_num, (client_padded_sequences, client_images, client_labels) in enumerate(client_datasets, 1):
        print(f"\n--- Client {client_num} starts training ---")

        # Create a client local model and load the weights of the global model
        local_model = create_model()
        local_model.set_weights(global_model.get_weights())

        # Compile the model, specify the optimizer and loss function
        local_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[CustomAccuracy()])

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

        # Get the weights of the initial model
        initial_weights = local_model.get_weights()

        # Divide the client data into training set and validation set
        client_X_train_text, client_X_val_text, client_X_train_images, client_X_val_images, client_y_train, client_y_val = train_test_split(
            client_padded_sequences, client_images, client_labels, test_size=0.2, random_state=client_num
        )

        # Make sure the data type is correct
        client_X_train_text = client_X_train_text.astype(np.int32)
        client_X_val_text = client_X_val_text.astype(np.int32)
        client_X_train_images = client_X_train_images.astype(np.float32)
        client_X_val_images = client_X_val_images.astype(np.float32)
        client_y_train = client_y_train.astype(np.float32)
        client_y_val = client_y_val.astype(np.float32)

        # Store validation data to evaluate the global model
        val_text_list.append(client_X_val_text)
        val_images_list.append(client_X_val_images)
        val_labels_list.append(client_y_val)
        # If it's the first round, evaluate the untrained model on the validation set
        if round_num == 1:
            initial_evaluation = local_model.evaluate(
                {'text_input': client_X_val_text, 'image_input': client_X_val_images},
                client_y_val,
                verbose=0
            )
            print(f"Client {client_num} initial (untrained) model evaluation metrics: {local_model.metrics_names} = {initial_evaluation}")
             # Calculate F1, Precision, Recall for the untrained model
            initial_val_predictions = local_model.predict({'text_input': client_X_val_text, 'image_input': client_X_val_images})
            initial_val_predictions_binary = np.where(initial_val_predictions > 0.5, 1, 0)
            initial_f1 = f1_score(client_y_val, initial_val_predictions_binary, average='weighted', zero_division=1)
            initial_recall = recall_score(client_y_val, initial_val_predictions_binary, average='weighted', zero_division=1)
            initial_precision = precision_score(client_y_val, initial_val_predictions_binary, average='weighted', zero_division=1)

            print(f"Client {client_num} initial (untrained) model - F1 Score: {initial_f1}, Recall: {initial_recall}, Precision: {initial_precision}")
        # Calculate F1, Precision, Recall for the untrained model
        initial_val_predictions = local_model.predict({'text_input': client_X_val_text, 'image_input': client_X_val_images})
        initial_val_predictions_binary = np.where(initial_val_predictions > 0.5, 1, 0)
        initial_f1 = f1_score(client_y_val, initial_val_predictions_binary, average='weighted', zero_division=1)
        initial_recall = recall_score(client_y_val, initial_val_predictions_binary, average='weighted', zero_division=1)
        initial_precision = precision_score(client_y_val, initial_val_predictions_binary, average='weighted', zero_division=1)

        print(f"Client {client_num} initial (untrained) model - F1 Score: {initial_f1}, Recall: {initial_recall}, Precision: {initial_precision}")
        # Calculate initial gradients
        initial_grads = compute_gradients(local_model, (client_X_train_text, client_X_train_images, client_y_train))

        # Define callback function
        client_model_filepath = f"client_{client_num}_round_{round_num}_model.h5"
        model_checkpoint = ModelCheckpoint(client_model_filepath, save_best_only=True, monitor='val_loss')
        

        # Train local model
        history = local_model.fit(
            {'text_input': client_X_train_text, 'image_input': client_X_train_images},
            client_y_train,
            epochs=10,
            batch_size=8,
            validation_data=({'text_input': client_X_val_text, 'image_input': client_X_val_images}, client_y_val),
            callbacks=[model_checkpoint],
            verbose=1
        )

        # Calculate the final gradient
        final_grads = compute_gradients(local_model, (client_X_train_text, client_X_train_images, client_y_train))

        # Calculate the gradient rate of change
        grad_change_rate = compute_gradient_change_rate(initial_grads, final_grads)
        print(f"Client {client_num} Gradient Change Rate: {grad_change_rate}")

        # Calculate weight difference
        final_weights = local_model.get_weights()
        weight_diff = [final - initial for final, initial in zip(final_weights, initial_weights)]

        #Store weight differences and gradient change rates
        client_weight_diffs_and_changes.append((weight_diff, grad_change_rate))

        # Evaluate local model
        client_evaluation = local_model.evaluate(
            {'text_input': client_X_val_text, 'image_input': client_X_val_images},
            client_y_val,
            verbose=0
        )
        client_metrics.append(client_evaluation)
        print(f"Client {client_num} Evaluation: {local_model.metrics_names} = {client_evaluation}")

        # Get the prediction results of the validation set
        val_predictions = local_model.predict({'text_input': client_X_val_text, 'image_input': client_X_val_images})
        val_predictions_binary = np.where(val_predictions > 0.5, 1, 0)

        # Calculate and print F1, Precision, Recall
        f1 = f1_score(client_y_val, val_predictions_binary, average='weighted', zero_division=1)
        recall = recall_score(client_y_val, val_predictions_binary, average='weighted', zero_division=1)
        precision = precision_score(client_y_val, val_predictions_binary, average='weighted', zero_division=1)

        print(f"Client {client_num} - F1 Score: {f1}, Recall: {recall}, Precision: {precision}")
        print(f"--- Client {client_num} training completed ---")

    # Server Aggregation
    print("\n*** Start aggregating client model weights ***")

    # Calculate the total gradient change rate
    total_grad_change = sum(grad_change_rate for _, grad_change_rate in client_weight_diffs_and_changes)

    # Aggregate weight difference
    new_weights = []
    for layer_idx in range(len(global_model.get_weights())):
        weighted_diff = np.zeros_like(global_model.get_weights()[layer_idx])
        for weight_diff, grad_change_rate in client_weight_diffs_and_changes:
            weight = grad_change_rate / (total_grad_change + 1e-7)
            weighted_diff += weight * weight_diff[layer_idx]
        # Update global weights
        new_weight = global_model.get_weights()[layer_idx] + weighted_diff
        new_weights.append(new_weight)

    # Update the weights of the global model
    global_model.set_weights(new_weights)
    print("*** Global model weight update completed ***")

    # Merge all client verification data
    combined_val_text = np.concatenate(val_text_list)
    combined_val_images = np.concatenate(val_images_list)
    combined_val_labels = np.concatenate(val_labels_list)

    # Make sure the data type is correct
    combined_val_text = combined_val_text.astype(np.int32)
    combined_val_images = combined_val_images.astype(np.float32)
    combined_val_labels = combined_val_labels.astype(np.float32)

    # Evaluate the global model
    global_evaluation = global_model.evaluate(
        {'text_input': combined_val_text, 'image_input': combined_val_images},
        combined_val_labels,
        verbose=0
    )
    global_metrics.append(global_evaluation)
    print(f"\n===== Evaluation metrics of the global model after {round_num} rounds: {global_model.metrics_names} = {global_evaluation} =====")

    # Save the global model
    global_model.save(f"global_model_round_{round_num}.h5")



===== 1 round of federated learning started =====

--- Client 1 starts training ---
Client 1 initial (untrained) model evaluation metrics: ['loss', 'custom_accuracy'] = [0.3137509226799011, 0.8786574602127075]
Client 1 initial (untrained) model - F1 Score: 0.0, Recall: 0.0, Precision: 1.0
Client 1 initial (untrained) model - F1 Score: 0.0, Recall: 0.0, Precision: 1.0


Calculating gradients: 100%|██████████| 249/249 [00:12<00:00, 19.66it/s]


Epoch 1/10
249/249 [==============================] - 7s 21ms/step - loss: 0.2901 - custom_accuracy: 0.8839 - val_loss: 0.2770 - val_custom_accuracy: 0.8911
Epoch 2/10
249/249 [==============================] - 5s 20ms/step - loss: 0.2714 - custom_accuracy: 0.8914 - val_loss: 0.2755 - val_custom_accuracy: 0.8900
Epoch 3/10
249/249 [==============================] - 5s 20ms/step - loss: 0.2782 - custom_accuracy: 0.8891 - val_loss: 0.3039 - val_custom_accuracy: 0.8824
Epoch 4/10
249/249 [==============================] - 5s 19ms/step - loss: 0.2913 - custom_accuracy: 0.8848 - val_loss: 0.2953 - val_custom_accuracy: 0.8824
Epoch 5/10
249/249 [==============================] - 4s 18ms/step - loss: 0.2921 - custom_accuracy: 0.8849 - val_loss: 0.2950 - val_custom_accuracy: 0.8831
Epoch 6/10
249/249 [==============================] - 4s 18ms/step - loss: 0.2904 - custom_accuracy: 0.8861 - val_loss: 0.3011 - val_custom_accuracy: 0.8807
Epoch 7/10
249/249 [==============================] - 4s 1

Calculating gradient change rate: 100%|██████████| 16/16 [00:00<00:00, 3999.81it/s]


Client 1 Gradient Change Rate: 1.197782327215538
Client 1 Evaluation: ['loss', 'custom_accuracy'] = [0.24187695980072021, 0.8997418284416199]
Client 1 - F1 Score: 0.394129632682432, Recall: 0.44208037825059104, Precision: 0.8092144724688053
--- Client 1 training completed ---

--- Client 2 starts training ---
Client 2 initial (untrained) model evaluation metrics: ['loss', 'custom_accuracy'] = [0.3113296329975128, 0.88080894947052]
Client 2 initial (untrained) model - F1 Score: 0.0, Recall: 0.0, Precision: 1.0
Client 2 initial (untrained) model - F1 Score: 0.0, Recall: 0.0, Precision: 1.0


...

Client 2 Gradient Change Rate: 1.5830360679656337
Client 2 Evaluation: ['loss', 'custom_accuracy'] = [0.13433560729026794, 0.9726046919822693]
Client 2 - F1 Score: 0.8768206303115124, Recall: 0.8664259927797834, Precision: 0.8974786527069227
--- Client 2 training completed ---

--- Client 3 starts training ---
Client 3 initial (untrained) model - F1 Score: 0.8806168007836475, Recall: 0.8652482269503546, Precision: 0.9061721785831368


Calculating gradients: 100%|██████████| 249/249 [00:13<00:00, 19.12it/s]


Epoch 1/10
249/249 [==============================] - 7s 21ms/step - loss: 0.0554 - custom_accuracy: 0.9767 - val_loss: 0.1068 - val_custom_accuracy: 0.9726
Epoch 2/10
249/249 [==============================] - 4s 18ms/step - loss: 0.0539 - custom_accuracy: 0.9764 - val_loss: 0.1148 - val_custom_accuracy: 0.9700
Epoch 3/10
249/249 [==============================] - 4s 18ms/step - loss: 0.0573 - custom_accuracy: 0.9768 - val_loss: 0.1181 - val_custom_accuracy: 0.9687
Epoch 4/10
249/249 [==============================] - 5s 18ms/step - loss: 0.0562 - custom_accuracy: 0.9762 - val_loss: 0.1088 - val_custom_accuracy: 0.9673
Epoch 5/10
249/249 [==============================] - 4s 18ms/step - loss: 0.0552 - custom_accuracy: 0.9755 - val_loss: 0.1162 - val_custom_accuracy: 0.9679
Epoch 6/10
249/249 [==============================] - 4s 18ms/step - loss: 0.0535 - custom_accuracy: 0.9775 - val_loss: 0.1138 - val_custom_accuracy: 0.9684
Epoch 7/10
249/249 [==============================] - 5s 1

Calculating gradient change rate: 100%|██████████| 16/16 [00:00<00:00, 8000.58it/s]


Client 3 Gradient Change Rate: 1.490795477556446
Client 3 Evaluation: ['loss', 'custom_accuracy'] = [0.10380537062883377, 0.9698795080184937]
Client 3 - F1 Score: 0.8668664868140871, Recall: 0.8546099290780141, Precision: 0.8917375866904246
--- Client 3 training completed ---

*** Start aggregating client model weights ***
*** Global model weight update completed ***

===== Evaluation metrics of the global model after 20 rounds: ['loss', 'custom_accuracy'] = [0.11913495510816574, 0.9731306433677673] =====


# Fedavg version

In [21]:
import tensorflow as tf

class CustomAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_accuracy', **kwargs):
        super(CustomAccuracy, self).__init__(name=name, **kwargs)
        self.total_correct = self.add_weight(name='total_correct', initializer='zeros', dtype=tf.float32)
        self.total_labels = self.add_weight(name='total_labels', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert the prediction results to binary
        y_pred_binary = tf.cast(y_pred > 0.5, tf.float32)

        # Condition 1: Label 0 is predicted as positive
        condition1 = y_pred_binary[:, 0] > 0.5  # shape: (batch_size,)

        # Condition 2: Any of the other labels is predicted to be positive
        condition2 = tf.reduce_any(y_pred_binary[:, 1:] > 0.5, axis=1)  # shape: (batch_size,)

        # Step 1: If condition 1 is true, set other labels to 0
        condition1_expanded = tf.expand_dims(condition1, axis=1)  # shape: (batch_size, 1)
        y_pred_modified = tf.where(
            condition1_expanded,
            tf.concat([y_pred_binary[:, 0:1], tf.zeros_like(y_pred_binary[:, 1:])], axis=1),
            y_pred_binary
        )

        # Step 2: If condition 1 is false and condition 2 is true, set label 0 to 0
        condition2_only = tf.logical_and(tf.logical_not(condition1), condition2)  # shape: (batch_size,)
        condition2_only_expanded = tf.expand_dims(condition2_only, axis=1)  # shape: (batch_size, 1)
        y_pred_modified = tf.where(
            condition2_only_expanded,
            tf.concat([tf.zeros_like(y_pred_binary[:, 0:1]), y_pred_modified[:, 1:]], axis=1),
            y_pred_modified
        )

        # **Make sure y_true and y_pred_modified are of the same type**
        y_true = tf.cast(y_true, tf.float32)
        y_pred_modified = tf.cast(y_pred_modified, tf.float32)

        # Compute the correct prediction for each label
        correct_predictions = tf.cast(tf.equal(y_pred_modified, y_true), tf.float32)

        # Calculate the total number of correct predictions for this batch
        correct_sum = tf.reduce_sum(correct_predictions)

        # Calculate the total number of labels in this batch
        labels_sum = tf.cast(tf.size(y_true), tf.float32)

        # Update the cumulative number of correct predictions and total number of labels
        self.total_correct.assign_add(correct_sum)
        self.total_labels.assign_add(labels_sum)

    def result(self):
        """
        Returns the accuracy, guaranteed to be between 0 and 1.
        """
        return self.total_correct / self.total_labels

    def reset_state(self):
        """
        Resets the accumulated number of correct predictions and total number of labels.
        """
        self.total_correct.assign(0.0)
        self.total_labels.assign(0.0)

In [22]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score

def create_model():
    # Text input and processing
    text_input = tf.keras.Input(shape=(100,), name='text_input')
    embedding_layer = tf.keras.layers.Embedding(input_dim=10000, output_dim=128)(text_input)
    lstm_layer = tf.keras.layers.LSTM(128)(embedding_layer)
    text_dense = tf.keras.layers.Dense(128, activation='relu')(lstm_layer)
    text_dropout = tf.keras.layers.Dropout(0.5)(text_dense)
    text_output = tf.keras.layers.Dense(64, activation='relu')(text_dropout)
    
    # Image input and processing
    image_input = tf.keras.Input(shape=(224, 224, 3), name='image_input')
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)
    base_model.trainable = False  # Freeze the pre-trained model
    image_pooling = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    image_dense = tf.keras.layers.Dense(128, activation='relu')(image_pooling)
    image_dropout = tf.keras.layers.Dropout(0.5)(image_dense)
    image_output = tf.keras.layers.Dense(64, activation='relu')(image_dropout)
    
    # Combine text and image features
    combined = tf.keras.layers.Concatenate()([text_output, image_output])
    combined_dense = tf.keras.layers.Dense(128, activation='relu')(combined)
    combined_dropout = tf.keras.layers.Dropout(0.5)(combined_dense)
    final_output = tf.keras.layers.Dense(14, activation='sigmoid')(combined_dropout)
    
    # Build model
    model = tf.keras.Model(inputs=[text_input, image_input], outputs=final_output)
    
    return model

# Initialize global model
global_model_weighted = create_model()

# Compile the global model for weighted average aggregation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
global_model_weighted.compile(optimizer=optimizer, loss='binary_crossentropy',  metrics=[CustomAccuracy()])

# Lists to store global metrics
global_metrics_weighted = []

# Number of federated learning rounds
num_rounds = 20


from sklearn.metrics import f1_score, recall_score, precision_score

# Start federated learning with uneven weighted average aggregation
for round_num in range(1, num_rounds + 1):
    print(f"\n===== Federated Learning Round {round_num} Begins =====")
    
    print("\n--- Training with Uneven Weighted Average Aggregation ---")

    client_weights_weighted = []
    client_metrics_weighted = []

    val_text_list = []
    val_images_list = []
    val_labels_list = []
  
    num_clients = len(client_datasets)
    client_weights_coefficients = [0.8] + [0.2 / (num_clients - 1)] * (num_clients - 1)
    
    for client_num, (client_padded_sequences, client_images, client_labels) in enumerate(client_datasets, 1):
        print(f"\n--- Client {client_num} Training Starts ---")

        local_model = create_model()
        local_model.set_weights(global_model_weighted.get_weights())

        local_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[CustomAccuracy()])
        
        # Split client data
        client_X_train_text, client_X_val_text, client_X_train_images, client_X_val_images, client_y_train, client_y_val = train_test_split(
            client_padded_sequences, client_images, client_labels, test_size=0.2, random_state=client_num
        )

        # Train local model
        history = local_model.fit(
            {'text_input': client_X_train_text, 'image_input': client_X_train_images},
            client_y_train,
            epochs=10,
            batch_size=8,
            validation_data=({'text_input': client_X_val_text, 'image_input': client_X_val_images}, client_y_val),
            verbose=1
        )
        
        client_weights_weighted.append(local_model.get_weights())

        # Predict on validation set
        val_predictions = local_model.predict({'text_input': client_X_val_text, 'image_input': client_X_val_images})
        val_predictions_binary = np.where(val_predictions > 0.5, 1, 0)

        # Calculate F1, Recall, Precision for each client with zero_division=1
        f1 = f1_score(client_y_val, val_predictions_binary, average='weighted', zero_division=1)
        recall = recall_score(client_y_val, val_predictions_binary, average='weighted', zero_division=1)
        precision = precision_score(client_y_val, val_predictions_binary, average='weighted', zero_division=1)
        
        # Get validation loss and accuracy from history object
        val_loss = history.history['val_loss'][-1]
        val_accuracy = history.history['val_custom_accuracy'][-1]  # Replace 'val_accuracy' with 'val_custom_accuracy'


        # Print metrics
        print(f"Client {client_num} - F1 Score: {f1}, Recall: {recall}, Precision: {precision}")
        print(f"Client {client_num} - Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")

        val_text_list.append(client_X_val_text)
        val_images_list.append(client_X_val_images)
        val_labels_list.append(client_y_val)
    
    # Aggregate client weights
    print("\n*** Aggregating Client Model Weights (Uneven Weighted Average) ***")
    new_weights = []
    for weights_list in zip(*client_weights_weighted):
        weighted_weights = np.zeros_like(weights_list[0])
        for weights, coeff in zip(weights_list, client_weights_coefficients):
            weighted_weights += weights * coeff
        new_weights.append(weighted_weights)
    
    # Update global model weights
    global_model_weighted.set_weights(new_weights)
    print("*** Global Model Weights Updated (Uneven Weighted Average) ***")

    combined_val_text = np.concatenate(val_text_list)
    combined_val_images = np.concatenate(val_images_list)
    combined_val_labels = np.concatenate(val_labels_list)

    global_evaluation = global_model_weighted.evaluate(
        {'text_input': combined_val_text, 'image_input': combined_val_images},
        combined_val_labels,
        verbose=0
    )
    global_metrics_weighted.append(global_evaluation)
    print(f"\n===== Global Model Evaluation after Round {round_num} (Uneven Weighted Average): {global_model_weighted.metrics_names} = {global_evaluation} =====")
    
    # Save
    global_model_weighted.save(f"global_model2_weighted_round_{round_num}.h5")



===== Federated Learning Round 1 Begins =====

--- Training with Uneven Weighted Average Aggregation ---

--- Client 1 Training Starts ---
Epoch 1/10
249/249 [==============================] - 7s 23ms/step - loss: 0.4817 - custom_accuracy: 0.8293 - val_loss: 0.3202 - val_custom_accuracy: 0.8779
Epoch 2/10
249/249 [==============================] - 5s 20ms/step - loss: 0.3495 - custom_accuracy: 0.8663 - val_loss: 0.2814 - val_custom_accuracy: 0.8835
Epoch 3/10
249/249 [==============================] - 5s 20ms/step - loss: 0.3088 - custom_accuracy: 0.8819 - val_loss: 0.2613 - val_custom_accuracy: 0.8900
Epoch 4/10
249/249 [==============================] - 5s 20ms/step - loss: 0.2773 - custom_accuracy: 0.8954 - val_loss: 0.2417 - val_custom_accuracy: 0.9015
Epoch 5/10
249/249 [==============================] - 5s 19ms/step - loss: 0.2630 - custom_accuracy: 0.8976 - val_loss: 0.2317 - val_custom_accuracy: 0.9038
Epoch 6/10
249/249 [==============================] - 5s 18ms/step - loss: 